# Conversion of COCO annotation JSON file to TFRecords

Given a COCO annotated JSON file, your goal is to convert it into a TFRecords  file necessary to train with the Mask RCNN model.

To accomplish this task, you will clone the TensorFlow Model Garden repo. The TensorFlow Model Garden is a repository with a number of different implementations of state-of-the-art (SOTA) models and modeling solutions for TensorFlow users.

This notebook is an end to end example. When you run the notebook, it will take COCO annotated JSON train and test files as an input and will convert them into TFRecord files. You can also output sharded TFRecord files in case your training and validation data is huge. It makes it easier for the algorithm to read and access the data.

**Note** - In this example, we assume that all our data is saved on Google drive and we will also write our outputs to Google drive. We also assume that the script will be used as a Google Colab notebook. But this can be changed according to the needs of users. They can modify this in case they are working on their local workstation, remote server or any other database. This colab notebook can be changed to a regular jupyter notebook running on a local machine according to the need of the users.

## Run the below command to connect to your google drive

In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
# import libraries
from google.colab import drive
import sys
from configparser import ConfigParser

In [ ]:
# "opencv-python-headless" version should be same of "opencv-python"
import pkg_resources
version_number = pkg_resources.get_distribution("opencv-python").version

!pip install opencv-python-headless==$version_number

     |████████████████████████████████| 21.8 MB 62.9 MB/s 


In [ ]:
# connect to google drive
drive.mount('/content/gdrive')

# making an alias for the root path
try:
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  print('Successful')
except Exception as e:
  print(e)
  print('Not successful')

Mounted at /content/gdrive
Successful


## Clone TensorFlow Model Garden repository

In [ ]:
# move to the specified folder where you want to clone
%cd $tensorflow_model_folder

/content/gdrive/My Drive/TFVision


In [ ]:
# clone the Model Garden directory for Tensorflow where all the config files and scripts are located for this project. 
# project folder name is - 'waste_identification_ml'
!git clone https://github.com/tensorflow/models.git 

In [ ]:
# Go to the model folder
%cd models

## **MUST CHANGE** - Import the path and parameters

In [ ]:
config = ConfigParser()

# path to the config file defining parameters
# config.ini file is an important file where all the parameter variables are located
# config.ini file resides in the 'config' folder which is in the 'pre_processing' folder
config.read('official/projects/waste_identification_ml/pre_processing/config/config.ini')

# path where you want to close tensorflow model directory
tensorflow_model_folder = config['tfrecord']['tensorflow_model_folder']
print(tensorflow_model_folder)

# path where you want to put the traiing tfrecord file
training_data_folder = config['tfrecord']['training_data_folder']
print(training_data_folder)

# path where you want to put the validation tfrecord file
validation_data_folder = config['tfrecord']['validation_data_folder']
print(validation_data_folder)

# path where all training images are located
training_images_folder = config['tfrecord']['training_images_folder']
print(training_images_folder)

# path of the training annotation file that needs to be converted
training_annotation_file = config['tfrecord']['training_annotation_file']
print(training_annotation_file)

# path where all validation images are located
validation_images_folder = config['tfrecord']['validation_images_folder']
print(validation_images_folder)

# path of the validation annotation file that needs tobe converted
validation_annotation_file = config['tfrecord']['validation_annotation_file']
print(validation_annotation_file)

/mydrive/TFVision/
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/train/
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/val/
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/Total_images/
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/_train.json
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/Total_images/
/mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/_val.json


## Create TFRecord for training data

In [ ]:
# create a folder for validation data
!mkdir -p $validation_data_folder

# create a folder for training data
!mkdir -p $training_data_folder

In [ ]:
# run the script to convert your json file to TFRecord file
# --num_shards (how many TFRecord sharded files you want)
!python3 -m official.vision.data.create_coco_tf_record --logtostderr \
      --image_dir=$training_images_folder \
      --object_annotations_file=$training_annotation_file \
      --output_file_prefix=$training_data_folder \
      --num_shards=100 \
      --include_masks=True \
      --num_processes=0

I0422 00:06:23.072771 139705362556800 create_coco_tf_record.py:494] writing to output path: /mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/train/
I0422 00:06:25.089654 139705362556800 create_coco_tf_record.py:366] Building bounding box index.
I0422 00:06:25.115955 139705362556800 create_coco_tf_record.py:377] 0 images are missing bboxes.
I0422 00:07:39.273266 139705362556800 tfrecord_lib.py:168] On image 0
I0422 00:09:03.214606 139705362556800 tfrecord_lib.py:168] On image 100
I0422 00:10:14.332473 139705362556800 tfrecord_lib.py:168] On image 200
I0422 00:11:11.556596 139705362556800 tfrecord_lib.py:168] On image 300
I0422 00:12:11.437826 139705362556800 tfrecord_lib.py:168] On image 400
I0422 00:13:13.166231 139705362556800 tfrecord_lib.py:168] On image 500
I0422 00:14:21.695016 139705362556800 tfrecord_lib.py:168] On image 600
I0422 00:15:24.191824 139705362556800 tfrecord_lib.py:168] On image 700
I0422 00:16:48.620902 139705362556800 tfrecord

## Create TFRecord for validation data

In [ ]:
# run the script to convert your json file to TFRecord file
# --num_shards (how many TFRecord sharded files you want)
!python3 -m official.vision.data.create_coco_tf_record --logtostderr \
      --image_dir=$validation_images_folder \
      --object_annotations_file=$validation_annotation_file \
      --output_file_prefix=$validation_data_folder \
      --num_shards=100 \
      --include_masks=True \
      --num_processes=0

I0421 20:53:39.071351 140304098097024 create_coco_tf_record.py:494] writing to output path: /mydrive/gtech/MRFs/Recykal/Latest_sharing_by_sanket/Google_Recykal/Taxonomy_version_2/val/
I0421 20:53:40.622877 140304098097024 create_coco_tf_record.py:366] Building bounding box index.
I0421 20:53:40.627101 140304098097024 create_coco_tf_record.py:377] 0 images are missing bboxes.
I0421 20:54:41.275259 140304098097024 tfrecord_lib.py:168] On image 0
I0421 20:56:53.052898 140304098097024 tfrecord_lib.py:168] On image 100
I0421 20:59:01.886727 140304098097024 tfrecord_lib.py:168] On image 200
I0421 21:01:12.356394 140304098097024 tfrecord_lib.py:168] On image 300
I0421 21:03:03.635432 140304098097024 tfrecord_lib.py:168] On image 400
I0421 21:05:04.787051 140304098097024 tfrecord_lib.py:168] On image 500
I0421 21:06:52.991898 140304098097024 tfrecord_lib.py:168] On image 600
I0421 21:09:02.626780 140304098097024 tfrecord_lib.py:168] On image 700
I0421 21:11:39.070799 140304098097024 tfrecord_l